In [1]:
import pathlib
from scripts.infer.structure import CrossDomainTypes4Py

dataset = CrossDomainTypes4Py(pathlib.Path("/nfs/data/students/bsparks/mdti4py-dataset-pool/cdt4py"))
artifact_root = pathlib.Path("/nfs/home/bsparks/mdti4py/datasets")

2023-09-28 22:06:34.185744: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 22:06:34.934709: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import experiments.inferred
entire_gt = experiments.inferred.load_groundtruths(artifact_root, dataset)

import experiments.predictions
trait_form = entire_gt.rename(columns={"adjusted_anno": "trait_gt_form"})
trait_form = trait_form[~trait_form.trait_gt_form.isin(["None", "Any"])]

ubiq_types = experiments.predictions.ubiquitous_types(trait_form)["trait_gt_form"]
common_types = experiments.predictions.common_types(trait_form)["trait_gt_form"]
rare_types = experiments.predictions.rare_types(trait_form)["trait_gt_form"]

/nfs/home/bsparks/mdti4py/datasets/CrossDomainTypes4Py/zwicker-group__numpy-py-pde/extended_ground_truth.csv: 100%|█████████████████████████████████████████████████████████████████████████████████████| 1548/1548 [00:19<00:00, 80.40it/s]
/nfs/home/bsparks/mdti4py/datasets/CrossDomainTypes4Py/zwicker-group__numpy-py-pde/context.csv: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1548/1548 [00:09<00:00, 156.01it/s]


In [3]:
validation_ars = list(map(lambda r: str(dataset.author_repo(r)), dataset.el_validation_set().keys()))
validation_gt = entire_gt[entire_gt["repository"].isin(validation_ars)]

In [4]:
test_ars = list(map(lambda r: str(dataset.author_repo(r)), dataset.el_test_set().keys()))
test_gt = entire_gt[entire_gt["repository"].isin(test_ars)]

In [5]:
del entire_gt
del trait_form

In [6]:
ubiq_types.value_counts()

trait_gt_form
str      50069
int      22605
bool     14107
float     8569
Dict      5495
Name: count, dtype: int64

In [7]:
common_types.value_counts()

trait_gt_form
ndarray               3926
List[str]             3851
List                  3236
Dict[str, Any]        2879
DataFrame             2483
                      ... 
Order                  104
Dict[str, Tuple]       102
Dict[int, List]        101
Token                  101
Callable[[], None]     100
Name: count, Length: 143, dtype: int64

In [8]:
rare_types.value_counts()

trait_gt_form
UID                                                                 99
Election                                                            99
File                                                                99
List[Token]                                                         99
Mock                                                                98
                                                                    ..
Union[EccKey, RsaKey]                                                1
Tuple[User, PublicKey, Union[Any], APIToken]                         1
Union[EccKey, RsaKey, str]                                           1
Tuple[User, PublicKey, Union[Any], PublicKeyVerificationRequest]     1
EventListeners                                                       1
Name: count, Length: 15401, dtype: int64

In [9]:
from experiments import pipeline, inferred
type4py_inferred = inferred.load_entire_inferred(
    artifact_root, 
    dataset, 
    tool_name="type4pyN1", 
    task="all"
)
typilus_inferred = inferred.load_entire_inferred(
    artifact_root,
    dataset,
    tool_name="typilusN1",
    task="all"
)
typet5_inferred = inferred.load_entire_inferred(
    artifact_root,
    dataset,
    tool_name="TypeT5TopN1",
    task="all"
)

/nfs/home/bsparks/mdti4py/datasets/CrossDomainTypes4Py/zwicker-group__numpy-py-pde/type4pyN1/all/inferred.csv: 100%|███████████████████████████████████████████████████████████████████████████████████| 1548/1548 [00:10<00:00, 148.84it/s]


Loaded 1538 inference artifacts


/nfs/home/bsparks/mdti4py/datasets/CrossDomainTypes4Py/zwicker-group__numpy-py-pde/typilusN1/all/inferred.csv: 100%|███████████████████████████████████████████████████████████████████████████████████| 1548/1548 [00:09<00:00, 158.91it/s]


Loaded 1541 inference artifacts


/nfs/home/bsparks/mdti4py/datasets/CrossDomainTypes4Py/zwicker-group__numpy-py-pde/TypeT5TopN1/all/inferred.csv: 100%|█████████████████████████████████████████████████████████████████████████████████| 1548/1548 [00:09<00:00, 162.40it/s]


Loaded 1482 inference artifacts


In [10]:
import experiments.probas

In [11]:
type4py_probas = experiments.probas.load_inferred_with_probablities(
    artifact_root,
    dataset,
    tool_name="type4py",
    task="all",
    inferred=type4py_inferred
)
typilus_probas = experiments.probas.load_inferred_with_probablities(
    artifact_root,
    dataset,
    tool_name="typilus",
    task="all",
    inferred=typilus_inferred
)
typet5_probas = experiments.probas.load_inferred_with_probablities(
    artifact_root,
    dataset,
    tool_name="typet5",
    task="all",
    inferred=typet5_inferred
)

Valid: 1166342 / 1166517
Annotated & Has Probability: 689355 / 689530
0 entries with a probability but no annotation
175 entries with an annotation but no probability
                                       repository  \
75    MarcSkovMadsen__numpy-data_models_analytics   
76    MarcSkovMadsen__numpy-data_models_analytics   
77    MarcSkovMadsen__numpy-data_models_analytics   
78    MarcSkovMadsen__numpy-data_models_analytics   
79    MarcSkovMadsen__numpy-data_models_analytics   
...                                           ...   
989                      yukinarit__flask-pyserde   
1001                     yukinarit__flask-pyserde   
1002                     yukinarit__flask-pyserde   
1006                     yukinarit__flask-pyserde   
1008                     yukinarit__flask-pyserde   

                                   file  category  \
75    api/tests/utils/test_etl_table.py  VARIABLE   
76    api/tests/utils/test_etl_table.py  VARIABLE   
77    api/tests/utils/test_etl_table.

In [12]:
import pandas as pd

def interpolate_average(df_with_some_probs: pd.DataFrame) -> pd.DataFrame:
    anno_no_prob_mask = df_with_some_probs["probability"].isna() & df_with_some_probs["anno"].notna()
    anno_no_prob = df_with_some_probs[anno_no_prob_mask]
    
    # display(anno_no_prob["anno"].value_counts())
    
    average_prob_per_anno = df_with_some_probs.groupby("anno").probability.mean()
    # display(average_prob_per_anno)
    
    average_prob_per_missing_anno = pd.merge(
        left=df_with_some_probs["anno"].value_counts(),
        right=average_prob_per_anno,
        left_index=True,
        right_index=True,
        how="inner",
    ).drop(columns=["count"])
    
    prob_dict = average_prob_per_missing_anno.to_dict()["probability"]
    df_with_some_probs['probability'].fillna(df_with_some_probs['anno'].map(prob_dict), inplace=True)
    
    still_missing_count = len(average_prob_per_missing_anno[average_prob_per_missing_anno.probability.isna()])
    if still_missing_count > 0:
        print("#Unique Annotations without Prob: ", still_missing_count, "; filling in blanks with 0.0")
        df_with_some_probs.loc[anno_no_prob_mask, "probability"].fillna(0, inplace=True)
    else:
        print("Successfully interpolated all values")
    return df_with_some_probs

In [13]:
type4py_inferred_with_probas = interpolate_average(type4py_probas)

Successfully interpolated all values


In [14]:
typilus_inferred_with_probas = interpolate_average(typilus_probas)

#Unique Annotations without Prob:  11 ; filling in blanks with 0.0


In [15]:
typet5_inferred_with_probas = interpolate_average(typet5_probas)

Successfully interpolated all values


In [16]:
type4py_test_adjusted = pipeline.factory(
    tool="type4pyN1", 
    groundtruth=test_gt, 
    inferred=type4py_inferred_with_probas, 
    form="adjusted"
).replace("<MISSING>", pd.NA)
type4py_test_base = pipeline.factory(
    tool="type4pyN1",
    groundtruth=test_gt,
    inferred=type4py_inferred_with_probas,
    form="base"
).replace("<MISSING>", pd.NA)

Initial prediction size: (1166517, 9)
Deriving limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166517/1166517 [00:09<00:00, 127554.41it/s]


Deriving adjusted form from limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166517/1166517 [00:08<00:00, 129742.88it/s]


Size after joining predictions to groundtruth: (223022, 13)
Reduced to evaluatable: (41562, 13)


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166517/1166517 [00:08<00:00, 134674.44it/s]


Deriving base form from adjusted form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166517/1166517 [00:09<00:00, 126315.83it/s]


(223022, 13)
Reduced to evaluatable: (41562, 13)


/nfs/home/bsparks/mdti4py/scripts/experiments/inferred.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned[clean_annos] = cleaned[clean_annos].fillna("<MISSING>")


In [17]:
typilus_test_adjusted = pipeline.factory(
    tool="typilusN1", 
    groundtruth=test_gt, 
    inferred=typilus_inferred_with_probas, 
    form="adjusted"
).replace("<MISSING>", pd.NA)
typilus_test_base = pipeline.factory(
    tool="typilusN1",
    groundtruth=test_gt,
    inferred=typilus_inferred_with_probas,
    form="base"
).replace("<MISSING>", pd.NA)

Initial prediction size: (1166481, 9)
Deriving limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166481/1166481 [00:09<00:00, 117070.59it/s]


Deriving adjusted form from limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166481/1166481 [00:10<00:00, 114206.10it/s]


Size after joining predictions to groundtruth: (213515, 13)
Reduced to evaluatable: (40630, 13)


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166481/1166481 [00:09<00:00, 117665.93it/s]


Deriving base form from adjusted form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166481/1166481 [00:09<00:00, 117413.68it/s]


(213515, 13)
Reduced to evaluatable: (40630, 13)


/nfs/home/bsparks/mdti4py/scripts/experiments/inferred.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned[clean_annos] = cleaned[clean_annos].fillna("<MISSING>")


In [18]:
typet5_test_adjusted = pipeline.factory(
    tool="TypeT5TopN1", 
    groundtruth=test_gt, 
    inferred=typet5_inferred_with_probas, 
    form="adjusted"
).replace("<MISSING>", pd.NA)
typet5_test_base = pipeline.factory(
    tool="TypeT5TopN1",
    groundtruth=test_gt,
    inferred=typet5_inferred_with_probas,
    form="base"
).replace("<MISSING>", pd.NA)

Initial prediction size: (1146586, 9)
Deriving limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1146586/1146586 [00:05<00:00, 224795.63it/s]


Deriving adjusted form from limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1146586/1146586 [00:05<00:00, 218935.02it/s]


Size after joining predictions to groundtruth: (123095, 13)
Reduced to evaluatable: (38413, 13)


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1146586/1146586 [00:05<00:00, 216202.20it/s]


Deriving base form from adjusted form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1146586/1146586 [00:05<00:00, 219732.88it/s]


(123095, 13)
Reduced to evaluatable: (38413, 13)


/nfs/home/bsparks/mdti4py/scripts/experiments/inferred.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned[clean_annos] = cleaned[clean_annos].fillna("<MISSING>")


# Measure Performance of Homogeneous Models upon Test Split

In [19]:
display(typilus_adj_test_perf := experiments.predictions.performance(
    typilus_test_adjusted.replace("<MISSING>", pd.NA),
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True,
))

display(typilus_base_test_perf := experiments.predictions.performance(
    typilus_test_base.replace("<MISSING>", pd.NA),
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True,
))

,observations,predictions,unassigned,matches,stracc,relacc
ubiquitous,18479,17559,920,12161,0.658098,0.692579
common,10378,9933,445,1704,0.164193,0.171549
rare,11773,11026,747,259,0.021999,0.023490
total,40630,38518,2112,14124,0.347625,0.366686


,observations,predictions,unassigned,matches,stracc,relacc
ubiquitous,18479,17559,920,12387,0.670328,0.705450
common,10378,9933,445,2784,0.268260,0.280278
rare,11773,11026,747,1252,0.106345,0.113550
total,40630,38518,2112,16423,0.404209,0.426372


In [20]:
display(t4py_adj_test_perf := experiments.predictions.performance(
    type4py_test_adjusted.replace("<MISSING>", pd.NA),
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True,
))

display(t4py_base_test_perf := experiments.predictions.performance(
    type4py_test_base.replace("<MISSING>", pd.NA),
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True,
))

,observations,predictions,unassigned,matches,stracc,relacc
ubiquitous,18871,17879,992,12929,0.685125,0.723139
common,10583,9717,866,3308,0.312577,0.340434
rare,12108,10680,1428,1314,0.108523,0.123034
total,41562,38276,3286,17551,0.422285,0.458538


,observations,predictions,unassigned,matches,stracc,relacc
ubiquitous,18871,17879,992,13058,0.691961,0.730354
common,10583,9717,866,3832,0.362090,0.394360
rare,12108,10680,1428,1719,0.141972,0.160955
total,41562,38276,3286,18609,0.447741,0.486179


In [21]:
display(tt5_adj_test_perf := experiments.predictions.performance(
    typet5_test_adjusted.replace("<MISSING>", pd.NA),
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True,
))

display(tt5_base_test_perf := experiments.predictions.performance(
    typet5_test_base.replace("<MISSING>", pd.NA),
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True,
))

,observations,predictions,unassigned,matches,stracc,relacc
ubiquitous,17903,15045,2858,13469,0.752332,0.895248
common,9623,8385,1238,6067,0.630469,0.723554
rare,10887,9472,1415,6259,0.574906,0.660790
total,38413,32902,5511,25795,0.671517,0.783995


,observations,predictions,unassigned,matches,stracc,relacc
ubiquitous,17903,15045,2858,13720,0.766352,0.911931
common,9623,8385,1238,6968,0.724099,0.831008
rare,10887,9472,1415,7061,0.648572,0.745460
total,38413,32902,5511,27749,0.722386,0.843383


# Join DataFrames for Voting Procedures

In [22]:
ignore = ["topn", "gt_anno", "context_category", "nested", "trait_gt_form"]
prediction_annos = ["anno_type4py", "anno_typilus", "anno_typet5"]
methods = ["method_type4py", "method_typilus", "method_typet5"]
probs = ["probability_type4py", "probability_typilus", "probability_typet5"]

In [23]:
aligned_adjusted = type4py_test_adjusted.drop(columns=ignore).merge(
    typilus_test_adjusted.drop(columns=ignore), 
    how="outer", 
    on=["repository", "category", "file", "qname", "qname_ssa"], 
    suffixes=("_type4py", "_typilus")
).merge(
    typet5_test_adjusted.drop(columns=ignore).rename(columns={"anno": "anno_typet5", "probability": "probability_typet5", "method": "method_typet5"}),
    how="outer",
    on=["repository", "category", "file", "qname", "qname_ssa"]
)

aligned_adjusted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41562 entries, 0 to 41561
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   file                 41562 non-null  object 
 1   category             41562 non-null  object 
 2   qname                41562 non-null  object 
 3   qname_ssa            41562 non-null  object 
 4   repository           41562 non-null  object 
 5   anno_type4py         38276 non-null  object 
 6   method_type4py       40688 non-null  object 
 7   probability_type4py  38276 non-null  float64
 8   anno_typilus         38518 non-null  object 
 9   method_typilus       39725 non-null  object 
 10  probability_typilus  38518 non-null  float64
 11  anno_typet5          32902 non-null  object 
 12  method_typet5        36761 non-null  object 
 13  probability_typet5   32902 non-null  float64
dtypes: float64(3), object(11)
memory usage: 4.4+ MB


In [24]:
aligned_base = type4py_test_base.drop(columns=ignore).merge(
    typilus_test_base.drop(columns=ignore), 
    how="outer", 
    on=["repository", "category", "file", "qname", "qname_ssa"], 
    suffixes=("_type4py", "_typilus")
).merge(
    typet5_test_base.drop(columns=ignore).rename(columns={"anno": "anno_typet5", "probability": "probability_typet5", "method": "method_typet5"}),
    how="outer",
    on=["repository", "category", "file", "qname", "qname_ssa"]
)

aligned_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41562 entries, 0 to 41561
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   file                 41562 non-null  object 
 1   category             41562 non-null  object 
 2   qname                41562 non-null  object 
 3   qname_ssa            41562 non-null  object 
 4   repository           41562 non-null  object 
 5   anno_type4py         38276 non-null  object 
 6   method_type4py       40688 non-null  object 
 7   probability_type4py  38276 non-null  float64
 8   anno_typilus         38518 non-null  object 
 9   method_typilus       39725 non-null  object 
 10  probability_typilus  38518 non-null  float64
 11  anno_typet5          32902 non-null  object 
 12  method_typet5        36761 non-null  object 
 13  probability_typet5   32902 non-null  float64
dtypes: float64(3), object(11)
memory usage: 4.4+ MB


In [25]:
joined_adjusted = experiments.inferred.join_truth_to_preds(
    truth=test_gt,
    predictions=aligned_adjusted,
    comparable_anno="adjusted_anno",
    prediction_annos=prediction_annos,
)

adjusted_eval = experiments.inferred.evaluatable(
    joined_adjusted, clean_annos=prediction_annos
).replace("<MISSING>", pd.NA)

/nfs/home/bsparks/mdti4py/scripts/experiments/inferred.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned[clean_annos] = cleaned[clean_annos].fillna("<MISSING>")


In [26]:
joined_base = experiments.inferred.join_truth_to_preds(
    truth=test_gt,
    predictions=aligned_base,
    comparable_anno="base_anno",
    prediction_annos=prediction_annos,
)
display(joined_base.columns)

base_eval = experiments.inferred.evaluatable(
    joined_base, clean_annos=prediction_annos
).replace("<MISSING>", pd.NA)

Index(['file', 'category', 'qname', 'qname_ssa', 'gt_anno', 'repository',
       'context_category', 'nested', 'trait_gt_form', 'anno_type4py',
       'method_type4py', 'probability_type4py', 'anno_typilus',
       'method_typilus', 'probability_typilus', 'anno_typet5', 'method_typet5',
       'probability_typet5'],
      dtype='object')

/nfs/home/bsparks/mdti4py/scripts/experiments/inferred.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned[clean_annos] = cleaned[clean_annos].fillna("<MISSING>")


In [27]:
import importlib
importlib.reload(experiments.predictions)
def hard_majority(df: pd.DataFrame) -> pd.DataFrame:
    predictions = df[prediction_annos]
    
    hm = predictions.mode(axis="columns")[0]
    # assert not hm.isna().any(), f"NAs: {hm.isna().sum()}"
    return pd.merge(
        left=df, 
        right=hm.rename("anno"), 
        left_index=True, 
        right_index=True,
    )

In [28]:
adjusted_hard_majority = hard_majority(adjusted_eval)

display(adjusted_hard_majority_perf := experiments.predictions.performance(
    adjusted_hard_majority,
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True
))

,observations,predictions,unassigned,matches,stracc,relacc
ubiquitous,18871,18165,706,14774,0.782894,0.813322
common,10583,10174,409,4926,0.465463,0.484175
rare,12108,11486,622,3656,0.301949,0.318301
total,41562,39825,1737,23356,0.561956,0.586466


In [66]:
base_hard_majority = hard_majority(base_eval)

display(base_hard_majority_perf := experiments.predictions.performance(
    base_hard_majority,
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True
))

,observations,predictions,unassigned,matches,stracc,relacc
ubiquitous,18871,18165,706,14975,0.793546,0.824388
common,10583,10174,409,6254,0.590948,0.614704
rare,12108,11486,622,4800,0.396432,0.417900
total,41562,39825,1737,26029,0.626269,0.653584


In [67]:
import collections

def soft_majority_vote(df: pd.DataFrame) -> pd.DataFrame:
    def soft_majority(row) -> str:
        weighted_votes = collections.defaultdict(float)
        for pred_col, prob_col in zip(prediction_annos, probs):
            # print(pred_col, type(row[pred_col]), prob_col, type(row[prob_col]))
            pred, prob = row[pred_col], row[prob_col]
            if not (pd.notna(pred) or prob == 0):
                continue
            weighted_votes[pred] += prob
        
        return max(weighted_votes, key=weighted_votes.get, default=pd.NA)
    
    sm = df.apply(soft_majority, axis=1)
    # assert not sm.isna().any(), f"NAs: {sm.isna().sum()}"
    return pd.merge(
        left=df, 
        right=sm.rename("anno"), 
        left_index=True, 
        right_index=True,
    )

In [68]:
adjusted_soft_majority = soft_majority_vote(adjusted_eval)
display(adjusted_soft_majority.columns)

display(adjusted_soft_majority[adjusted_soft_majority.anno.isna()].anno.count())

display(adjusted_soft_majority_perf := experiments.predictions.performance(
    adjusted_soft_majority,
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True
))

Index(['file', 'category', 'qname', 'qname_ssa', 'gt_anno', 'repository',
       'context_category', 'nested', 'trait_gt_form', 'anno_type4py',
       'method_type4py', 'probability_type4py', 'anno_typilus',
       'method_typilus', 'probability_typilus', 'anno_typet5', 'method_typet5',
       'probability_typet5', 'anno'],
      dtype='object')

0

,observations,predictions,unassigned,matches,stracc,relacc
ubiquitous,18871,18165,706,15763,0.835303,0.867768
common,10583,10174,409,6432,0.607767,0.632200
rare,12108,11486,622,5955,0.491824,0.518457
total,41562,39825,1737,28150,0.677301,0.706842


In [69]:
base_soft_majority = soft_majority_vote(base_eval)

display(base_soft_majority_perf := experiments.predictions.performance(
    base_soft_majority,
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True
))

,observations,predictions,unassigned,matches,stracc,relacc
ubiquitous,18871,18165,706,16000,0.847862,0.880815
common,10583,10174,409,7414,0.700557,0.728720
rare,12108,11486,622,6928,0.572184,0.603169
total,41562,39825,1737,30342,0.730042,0.761883


In [70]:
identical_cols = ["predictions", "observations", "unassigned"]
pred_occ = adjusted_hard_majority_perf[["observations", "predictions"]]

disp_adj_hm = adjusted_hard_majority_perf.drop(columns=identical_cols)
disp_adj_sm = adjusted_soft_majority_perf.drop(columns=identical_cols)

display(disp_adj_voting := pd.concat(
    [pred_occ, disp_adj_hm, disp_adj_sm],
    keys=["Statistics", "Hard Majority", "Soft Majority"],
    axis=1
))

pred_occ = base_hard_majority_perf[["observations", "predictions"]]

disp_base_hm = base_hard_majority_perf.drop(columns=identical_cols)
disp_base_sm = base_soft_majority_perf.drop(columns=identical_cols)

display(disp_base_voting := pd.concat(
    [pred_occ, disp_base_hm, disp_base_sm],
    keys=["Statistics", "Hard Majority", "Soft Majority"],
    axis=1
))

Statistics             Hard Majority                      \
           observations predictions       matches    stracc    relacc   
ubiquitous        18871       18165         14774  0.782894  0.813322   
common            10583       10174          4926  0.465463  0.484175   
rare              12108       11486          3656  0.301949  0.318301   
total             41562       39825         23356  0.561956  0.586466   

           Soft Majority                      
                 matches    stracc    relacc  
ubiquitous         15763  0.835303  0.867768  
common              6432  0.607767  0.632200  
rare                5955  0.491824  0.518457  
total              28150  0.677301  0.706842

Statistics             Hard Majority                      \
           observations predictions       matches    stracc    relacc   
ubiquitous        18871       18165         14975  0.793546  0.824388   
common            10583       10174          6254  0.590948  0.614704   
rare              12108       11486          4800  0.396432  0.417900   
total             41562       39825         26029  0.626269  0.653584   

           Soft Majority                      
                 matches    stracc    relacc  
ubiquitous         16000  0.847862  0.880815  
common              7414  0.700557  0.728720  
rare                6928  0.572184  0.603169  
total              30342  0.730042  0.761883

In [71]:
display(disp_voting := pd.concat(
    [
        pd.concat([pred_occ], keys=["Statistics"], axis=1),
        pd.concat([disp_adj_hm, disp_base_hm], keys=["adjusted", "base"], axis=1),
        pd.concat([disp_adj_sm, disp_base_sm], keys=["adjusted", "base"], axis=1),
    ],
    keys=["", "Hard Majority", "Soft Majority"], 
    axis=1
))

print(disp_voting.to_latex(
    float_format="%.2f",
    label="tbl:voting_ml_perf",
    caption=("Performance of Voting Procedures upon adjusted \& base canonical forms")
))

Hard Majority                              \
             Statistics                  adjusted                        base   
           observations predictions       matches    stracc    relacc matches   
ubiquitous        18871       18165         14774  0.782894  0.813322   14975   
common            10583       10174          4926  0.465463  0.484175    6254   
rare              12108       11486          3656  0.301949  0.318301    4800   
total             41562       39825         23356  0.561956  0.586466   26029   

                               Soft Majority                              \
                                    adjusted                        base   
              stracc    relacc       matches    stracc    relacc matches   
ubiquitous  0.793546  0.824388         15763  0.835303  0.867768   16000   
common      0.590948  0.614704          6432  0.607767  0.632200    7414   
rare        0.396432  0.417900          5955  0.491824  0.518457    6928   
total       0.626269  0.653584         28150  0.677301  0.706842   30342   

                                
                                
              stracc    relacc  
ubiquitous  0.847862  0.880815  
common      0.700557  0.728720  
rare        0.572184  0.603169  
total       0.730042  0.761883

\begin{table}
\caption{Performance of Voting Procedures upon adjusted \& base canonical forms}
\label{tbl:voting_ml_perf}
\begin{tabular}{lrrrrrrrrrrrrrr}
\toprule
 & \multicolumn{2}{r}{} & \multicolumn{6}{r}{Hard Majority} & \multicolumn{6}{r}{Soft Majority} \\
 & \multicolumn{2}{r}{Statistics} & \multicolumn{3}{r}{adjusted} & \multicolumn{3}{r}{base} & \multicolumn{3}{r}{adjusted} & \multicolumn{3}{r}{base} \\
 & observations & predictions & matches & stracc & relacc & matches & stracc & relacc & matches & stracc & relacc & matches & stracc & relacc \\
\midrule
ubiquitous & 18871 & 18165 & 14774 & 0.78 & 0.81 & 14975 & 0.79 & 0.82 & 15763 & 0.84 & 0.87 & 16000 & 0.85 & 0.88 \\
common & 10583 & 10174 & 4926 & 0.47 & 0.48 & 6254 & 0.59 & 0.61 & 6432 & 0.61 & 0.63 & 7414 & 0.70 & 0.73 \\
rare & 12108 & 11486 & 3656 & 0.30 & 0.32 & 4800 & 0.40 & 0.42 & 5955 & 0.49 & 0.52 & 6928 & 0.57 & 0.60 \\
total & 41562 & 39825 & 23356 & 0.56 & 0.59 & 26029 & 0.63 & 0.65 & 28150 & 0.68 & 0.71 & 3034

In [76]:
per_model_columns = ["observations", "predictions", "matches", "stracc", "relacc"]
per_prediction_columns = ["observations", "predictions"]
per_form_columns = ["stracc", "relacc"]

In [77]:
typilus_test_perf = pd.concat(
    [
        typilus_adj_test_perf[per_prediction_columns],
        typilus_adj_test_perf[per_form_columns],
        typilus_base_test_perf[per_form_columns]
    ],
    keys=["", "adjusted", "base"],
    axis=1,
)

t4py_test_perf = pd.concat(
    [
        t4py_adj_test_perf[per_prediction_columns],
        t4py_adj_test_perf[per_form_columns],
        t4py_base_test_perf[per_form_columns]
    ],
    keys=["", "adjusted", "base"],
    axis=1,
)

tt5_test_perf = pd.concat(
    [
        tt5_adj_test_perf[per_prediction_columns],
        tt5_adj_test_perf[per_form_columns],
        tt5_base_test_perf[per_form_columns]
    ],
    keys=["", "adjusted", "base"],
    axis=1,
)

hard_majority_perf = pd.concat(
    [
        adjusted_hard_majority_perf[per_prediction_columns],
        adjusted_hard_majority_perf[per_form_columns],
        base_hard_majority_perf[per_form_columns]
    ],
    keys=["", "adjusted", "base"],
    axis=1,
)
soft_majority_perf = pd.concat(
    [
        adjusted_soft_majority_perf[per_prediction_columns],
        adjusted_soft_majority_perf[per_form_columns],
        base_soft_majority_perf[per_form_columns]
    ],
    keys=["", "adjusted", "base"],
    axis=1,
)

In [99]:
by_majority_model_scarcity = pd.concat(
    [tt5_test_perf, hard_majority_perf, soft_majority_perf],
    keys=["TypeT5", "HM(TT5,Typ,T4P)", "SM(TT5,Typ,T4P)"],
)
by_majority_model_scarcity.index = by_majority_model_scarcity.index.set_names(["model", "scarcity"])

by_majority_model_scarcity = pd.concat([g for _, g in by_majority_model_scarcity.swaplevel().groupby("scarcity", sort=False)])
display(by_majority_model_scarcity)

adjusted            \
                           observations predictions    stracc    relacc   
scarcity   model                                                          
ubiquitous TypeT5                 17903       15045  0.752332  0.895248   
           HM(TT5,Typ,T4P)        18871       18165  0.782894  0.813322   
           SM(TT5,Typ,T4P)        18871       18165  0.835303  0.867768   
common     TypeT5                  9623        8385  0.630469  0.723554   
           HM(TT5,Typ,T4P)        10583       10174  0.465463  0.484175   
           SM(TT5,Typ,T4P)        10583       10174  0.607767  0.632200   
rare       TypeT5                 10887        9472  0.574906  0.660790   
           HM(TT5,Typ,T4P)        12108       11486  0.301949  0.318301   
           SM(TT5,Typ,T4P)        12108       11486  0.491824  0.518457   
total      TypeT5                 38413       32902  0.671517  0.783995   
           HM(TT5,Typ,T4P)        41562       39825  0.561956  0.586466   
           SM(TT5,Typ,T4P)        41562       39825  0.677301  0.706842   

                                base            
                              stracc    relacc  
scarcity   model                                
ubiquitous TypeT5           0.766352  0.911931  
           HM(TT5,Typ,T4P)  0.793546  0.824388  
           SM(TT5,Typ,T4P)  0.847862  0.880815  
common     TypeT5           0.724099  0.831008  
           HM(TT5,Typ,T4P)  0.590948  0.614704  
           SM(TT5,Typ,T4P)  0.700557  0.728720  
rare       TypeT5           0.648572  0.745460  
           HM(TT5,Typ,T4P)  0.396432  0.417900  
           SM(TT5,Typ,T4P)  0.572184  0.603169  
total      TypeT5           0.722386  0.843383  
           HM(TT5,Typ,T4P)  0.626269  0.653584  
           SM(TT5,Typ,T4P)  0.730042  0.761883

In [100]:
print(by_majority_model_scarcity.to_latex(
    float_format="{:.2f}".format,
))

\begin{tabular}{llrrrrrr}
\toprule
 &  & \multicolumn{2}{r}{} & \multicolumn{2}{r}{adjusted} & \multicolumn{2}{r}{base} \\
 &  & observations & predictions & stracc & relacc & stracc & relacc \\
scarcity & model &  &  &  &  &  &  \\
\midrule
\multirow[t]{3}{*}{ubiquitous} & TypeT5 & 17903 & 15045 & 0.75 & 0.90 & 0.77 & 0.91 \\
 & HM(TT5,Typ,T4P) & 18871 & 18165 & 0.78 & 0.81 & 0.79 & 0.82 \\
 & SM(TT5,Typ,T4P) & 18871 & 18165 & 0.84 & 0.87 & 0.85 & 0.88 \\
\cline{1-8}
\multirow[t]{3}{*}{common} & TypeT5 & 9623 & 8385 & 0.63 & 0.72 & 0.72 & 0.83 \\
 & HM(TT5,Typ,T4P) & 10583 & 10174 & 0.47 & 0.48 & 0.59 & 0.61 \\
 & SM(TT5,Typ,T4P) & 10583 & 10174 & 0.61 & 0.63 & 0.70 & 0.73 \\
\cline{1-8}
\multirow[t]{3}{*}{rare} & TypeT5 & 10887 & 9472 & 0.57 & 0.66 & 0.65 & 0.75 \\
 & HM(TT5,Typ,T4P) & 12108 & 11486 & 0.30 & 0.32 & 0.40 & 0.42 \\
 & SM(TT5,Typ,T4P) & 12108 & 11486 & 0.49 & 0.52 & 0.57 & 0.60 \\
\cline{1-8}
\multirow[t]{3}{*}{total} & TypeT5 & 38413 & 32902 & 0.67 & 0.78 & 0.72 & 0.84

In [97]:
by_scarcity_model = pd.concat(
    [typilus_test_perf, t4py_test_perf, tt5_test_perf, hard_majority_perf, soft_majority_perf],
    keys=["Typilus", "Type4Py", "TypeT5", "HM(TT5,Typ,T4P)", "SM(TT5,Typ,T4P)"],
)
by_scarcity_model.index = by_scarcity_model.index.set_names(["model", "scarcity"])

by_scarcity_model = pd.concat([g for _, g in by_scarcity_model.swaplevel().groupby("scarcity", sort=False)])
display(by_scarcity_model)

adjusted            \
                           observations predictions    stracc    relacc   
scarcity   model                                                          
ubiquitous Typilus                18479       17559  0.658098  0.692579   
           Type4Py                18871       17879  0.685125  0.723139   
           TypeT5                 17903       15045  0.752332  0.895248   
           HM(TT5,Typ,T4P)        18871       18165  0.782894  0.813322   
           SM(TT5,Typ,T4P)        18871       18165  0.835303  0.867768   
common     Typilus                10378        9933  0.164193  0.171549   
           Type4Py                10583        9717  0.312577  0.340434   
           TypeT5                  9623        8385  0.630469  0.723554   
           HM(TT5,Typ,T4P)        10583       10174  0.465463  0.484175   
           SM(TT5,Typ,T4P)        10583       10174  0.607767  0.632200   
rare       Typilus                11773       11026  0.021999  0.023490   
           Type4Py                12108       10680  0.108523  0.123034   
           TypeT5                 10887        9472  0.574906  0.660790   
           HM(TT5,Typ,T4P)        12108       11486  0.301949  0.318301   
           SM(TT5,Typ,T4P)        12108       11486  0.491824  0.518457   
total      Typilus                40630       38518  0.347625  0.366686   
           Type4Py                41562       38276  0.422285  0.458538   
           TypeT5                 38413       32902  0.671517  0.783995   
           HM(TT5,Typ,T4P)        41562       39825  0.561956  0.586466   
           SM(TT5,Typ,T4P)        41562       39825  0.677301  0.706842   

                                base            
                              stracc    relacc  
scarcity   model                                
ubiquitous Typilus          0.670328  0.705450  
           Type4Py          0.691961  0.730354  
           TypeT5           0.766352  0.911931  
           HM(TT5,Typ,T4P)  0.793546  0.824388  
           SM(TT5,Typ,T4P)  0.847862  0.880815  
common     Typilus          0.268260  0.280278  
           Type4Py          0.362090  0.394360  
           TypeT5           0.724099  0.831008  
           HM(TT5,Typ,T4P)  0.590948  0.614704  
           SM(TT5,Typ,T4P)  0.700557  0.728720  
rare       Typilus          0.106345  0.113550  
           Type4Py          0.141972  0.160955  
           TypeT5           0.648572  0.745460  
           HM(TT5,Typ,T4P)  0.396432  0.417900  
           SM(TT5,Typ,T4P)  0.572184  0.603169  
total      Typilus          0.404209  0.426372  
           Type4Py          0.447741  0.486179  
           TypeT5           0.722386  0.843383  
           HM(TT5,Typ,T4P)  0.626269  0.653584  
           SM(TT5,Typ,T4P)  0.730042  0.761883

In [98]:
print(by_scarcity_model.to_latex(
    float_format="{:.2f}".format,
))

\begin{tabular}{llrrrrrr}
\toprule
 &  & \multicolumn{2}{r}{} & \multicolumn{2}{r}{adjusted} & \multicolumn{2}{r}{base} \\
 &  & observations & predictions & stracc & relacc & stracc & relacc \\
scarcity & model &  &  &  &  &  &  \\
\midrule
\multirow[t]{5}{*}{ubiquitous} & Typilus & 18479 & 17559 & 0.66 & 0.69 & 0.67 & 0.71 \\
 & Type4Py & 18871 & 17879 & 0.69 & 0.72 & 0.69 & 0.73 \\
 & TypeT5 & 17903 & 15045 & 0.75 & 0.90 & 0.77 & 0.91 \\
 & HM(TT5,Typ,T4P) & 18871 & 18165 & 0.78 & 0.81 & 0.79 & 0.82 \\
 & SM(TT5,Typ,T4P) & 18871 & 18165 & 0.84 & 0.87 & 0.85 & 0.88 \\
\cline{1-8}
\multirow[t]{5}{*}{common} & Typilus & 10378 & 9933 & 0.16 & 0.17 & 0.27 & 0.28 \\
 & Type4Py & 10583 & 9717 & 0.31 & 0.34 & 0.36 & 0.39 \\
 & TypeT5 & 9623 & 8385 & 0.63 & 0.72 & 0.72 & 0.83 \\
 & HM(TT5,Typ,T4P) & 10583 & 10174 & 0.47 & 0.48 & 0.59 & 0.61 \\
 & SM(TT5,Typ,T4P) & 10583 & 10174 & 0.61 & 0.63 & 0.70 & 0.73 \\
\cline{1-8}
\multirow[t]{5}{*}{rare} & Typilus & 11773 & 11026 & 0.02 & 0.02 & 0.11 